In [3]:
import pandas as pd
import numpy as numpy
import requests
from bs4 import BeautifulSoup


In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
html = response.content
scraped = BeautifulSoup(html, 'html.parser')

In [5]:
scraped.h1

<h1 class="firstHeading" id="firstHeading" lang="en">List of postal codes of Canada: M</h1>

In [6]:
title_text = scraped.title.text.strip()
print(title_text)

List of postal codes of Canada: M - Wikipedia


In [56]:
# Create the table
table = scraped.find('table', attrs = {'class':'wikitable sortable'})

# Get the headers
table_head = table.findAll('th')
for i, head in enumerate(table_head): table_head[i]=str(table_head[i]).replace("<th>","").replace("</th>","").replace("\n","")

In [57]:
print(table_head)


['Postal Code', 'Borough', 'Neighbourhood']


In [73]:
rows = table.findAll('tr')
rows = rows[1:len(rows)]
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

df = pd.DataFrame(rows)
df[table_head] = df[0].str.split("\n</td>\n<td>", n = 2, expand = True)
df.drop(columns=[0],inplace =True)

In [74]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove rows from the table where Borough is 'Not assigned'

In [60]:
# Remove rows from the table where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']


In [69]:
df1.head()
df1.shape

(103, 3)

### Combine Neighbourhoods which share the same Postal Code

In [70]:
df2 = df1.groupby(['Postal Code', 'Borough'], sort = False).agg(', '.join)
df2.reset_index(inplace=True)
df2.head(20)
df2.shape

(103, 3)

In [72]:
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned', df2['Borough'], df2['Neighbourhood'])
df2.head()


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
